In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
### Load the trained model, scaler pickle, onehot
model=load_model('artifacts/model_salary.h5')

## load the encoder and scaler
with open('artifacts/onehot_encoder_geo.pkl','rb') as file: # rb = read bytes
    onehot_encoder_geo = pickle.load(file)

with open('artifacts/onehot_encoder_gender.pkl', 'rb') as file:
    onehot_encoder_gender = pickle.load(file)

with open('artifacts/scaler_salary.pkl', 'rb') as file:
    scaler = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'Exited': 1
}

In [4]:
# One-hot encode 'Geography'
geo_encoded = onehot_encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

# One-hot encode 'Gender'
gender_encoded = onehot_encoder_gender.transform([[input_data['Gender']]]).toarray()
gender_encoded_df = pd.DataFrame(gender_encoded, columns=onehot_encoder_gender.get_feature_names_out(['Gender']))

print(geo_encoded_df)
print(gender_encoded_df)

   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0
   Gender_Female  Gender_Male
0            0.0          1.0


d:\Work & Study\Study & Projects\Customer Churn Prediction\venv_cus_churn\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(
d:\Work & Study\Study & Projects\Customer Churn Prediction\venv_cus_churn\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited
0,600,France,Male,40,3,60000,2,1,1,1


In [6]:
# Drop the original 'Geography' and 'Gender' columns from input_df
input_df = input_df.drop(['Geography', 'Gender'], axis=1)

# Concatenate the one-hot encoded DataFrames with the original input_df
input_df = pd.concat([input_df, gender_encoded_df, geo_encoded_df], axis=1)

input_df

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,600,40,3,60000,2,1,1,1,0.0,1.0,1.0,0.0,0.0


In [7]:
# Reorder the columns of input_df to match the order during the fitting of the scaler
column_order = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'Exited', 'Gender_Female', 'Gender_Male', 'Geography_France', 'Geography_Germany', 'Geography_Spain']
input_df = input_df[column_order]

# Scaling the input data
input_scaled = scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.10479359, -0.69539349, -0.25781119,  0.80843615,
         0.64920267,  0.97481699,  1.96626042, -0.91324755,  0.91324755,
         1.00150113, -0.57946723, -0.57638802]])

In [8]:
## Predict churn
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step


array([[83505.44]], dtype=float32)

In [9]:
prediction_proba = prediction[0][0]
prediction_proba

83505.44

In [10]:
if prediction_proba > 0.5:
    print('The customer is likely to churn.')
else:
    print('The customer is not likely to churn.')

The customer is likely to churn.


In [1]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.16.1
3.8.0


In [1]:
import sklearn
print(sklearn.__version__)

1.6.1
